<h3>Import libraries & packages</h3>

In [23]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical

<h3>Load data</h3>

In [17]:
data = []
labels = []

PATH_DATA = "./SMILEsmileD/SMILEs/"

directory_positive = os.fsencode(PATH_DATA + "positives/positives7/")

for file in os.listdir(directory_positive):
    filename = os.fsdecode(file)
    img = cv2.cvtColor(cv2.imread(PATH_DATA + "positives/positives7/" + filename), \
                       cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (28, 28))
    img = img_to_array(img)
    data.append(img)
    
    label = "positive"
    labels.append(label)

directory_negative = os.fsencode(PATH_DATA + "negatives/negatives7/")

for file in os.listdir(directory_negative):
    filename = os.fsdecode(file)
    img = cv2.cvtColor(cv2.imread(PATH_DATA + "negatives/negatives7/" + filename), \
                       cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (28, 28))
    img = img_to_array(img)
    data.append(img)
    
    label = "negative"
    labels.append(label)

<h3>Data preprocessing</h3>

In [30]:
# Scale the raw pxel intensities to the range [0, 1]
data = np.array(data, dtype=np.float32) / 255.0
labels = np.array(labels)

# Convert the labels from strings to vectors
le = LabelEncoder().fit(labels)
labels = to_categorical(le.transform(labels), 2) # one hot vector